In [ ]:
!pip install selenium

In [ ]:
!sudo apt-get update
!sudo apt install chromium-chromedriver

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import Select
import re
import numpy as np
import pandas as pd
import time

In [2]:
# will resolve driver compatibility issues
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [3]:
def extract_links_from_page(driver):
    links = []

    # Find all devices in the grid with class name "box--ujueT" using By.CLASS_NAME
    devices = driver.find_elements(By.CLASS_NAME, 'box--ujueT')

    # Loop through each device and get the href links using the id "id-a-link"
    for device in devices:
        # Find all elements with class "gridItem--Yd0sa" inside each device
        grid_items = device.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

        # Loop through each grid item to get the href link
        for grid_item in grid_items:
            # Find the anchor element with id "id-a-link" inside each grid item
            anchor_element = grid_item.find_element(By.ID, "id-a-link")

            # Get the href attribute value
            href_link = anchor_element.get_attribute("href")

            # Append the href link to the list
            links.append(href_link)

    return links

In [29]:
def extract_links_for_pages(driver, links_list, end_page, brand):
    # URL pattern for the pages
    url_pattern = "https://www.daraz.pk/smartphones/" + brand + "/?page={}"

    # Extract links from each page in the specified range
    for page_number in range(1, end_page + 1):
        # Construct the URL for the current page
        current_url = url_pattern.format(page_number)

        # Open the URL
        driver.get(current_url)

        # Extract links from the current page
        links_list.extend(extract_links_from_page(driver))

In [30]:
driver = web_driver()

In [32]:
all_links = []
brands = ['samsung', 'vivo', 'infinix', 'realme', 'tecno']
end_pages = [14, 8, 11, 6, 7]


for i in range(5):
  extract_links_for_pages(driver, all_links, end_pages[i], brands[i])

In [33]:
all_links = list(set(all_links))

In [34]:
len(all_links)

890

In [35]:
links_dict = {'link': all_links}
print(links_dict)
df = pd.DataFrame.from_dict(links_dict)
df.to_csv("links.csv", index = False)

{'link': ['https://www.daraz.pk/products/y02-t-4-gb-64-gb-5-mp-8-mp-i427595776-s2031601043.html?search=1', 'https://www.daraz.pk/products/6gb-6gb-y17s-128-664-5000mah-i434277493-s2082823249.html?search=1', 'https://www.daraz.pk/products/a14-6gb-128-gb-pta-1-i428260915-s2035598256.html?search=1', 'https://www.daraz.pk/products/10-4gb-128gb-hd-90hz-66-mah-18-8mp-50mp-pta-i435539809-s2089987414.html?search=1', 'https://www.daraz.pk/products/20-i-4-gb-128-8-mp-13-mp-af-i419714386-s1984722187.html?search=1', 'https://www.daraz.pk/products/y-16-4-gb-64-gb-5-mp-13-mp-2-mp-5000-mah-i385227136-s1905556521.html?search=1', 'https://www.daraz.pk/products/4gb-a04-64gb-50mp-5000mah-i432610944-s2065367155.html?search=1', 'https://www.daraz.pk/products/8gb-a24-1-000-gb-128-66-mah-i424281029-s2010272727.html?search=1', 'https://www.daraz.pk/products/5-lte-652-2-gb-32-gb-1-pta-i261022192-s1879180217.html?search=1', 'https://www.daraz.pk/products/tecno-spark-8c-66-inch-display-4gb-ram-128gb-rom-i42252726

In [36]:
df = pd.read_csv("links.csv")
df['link'].head()


0    https://www.daraz.pk/products/y02-t-4-gb-64-gb...
1    https://www.daraz.pk/products/6gb-6gb-y17s-128...
2    https://www.daraz.pk/products/a14-6gb-128-gb-p...
3    https://www.daraz.pk/products/10-4gb-128gb-hd-...
4    https://www.daraz.pk/products/20-i-4-gb-128-8-...
Name: link, dtype: object

In [37]:
def extract_comments_from_page(driver):
    # Extract comments from the current page
    review_items = driver.find_elements(By.CLASS_NAME, 'review-item')
    comments = [review_item.find_element(By.CLASS_NAME, 'review-content-sl').text for review_item in review_items]
    return comments

In [10]:
def is_next_button_disabled(next_button):
    # Check if the "Next" button is disabled
    return "ant-pagination-disabled" in next_button.get_attribute("class")

In [38]:
def get_mobile_details(driver, link):
    # Open the URL
    driver.get(link)

    # Initialize variables to store details
    mobile_name = None
    price = None
    rating = None
    comments = []
    seller_name = None
    positive_seller_ratings = None
    ship_on_time = None
    chat_response_rate = None

    try:
        # Try to extract the mobile name
        mobile_name = driver.find_element(By.CLASS_NAME, 'pdp-mod-product-badge-title').text
    except NoSuchElementException:
        pass

    try:
        # Try to extract the price
        price = driver.find_element(By.CLASS_NAME, 'pdp-price.pdp-price_type_normal').text
    except NoSuchElementException:
        pass

    try:
        # Try to extract the rating
        rating = driver.find_element(By.CLASS_NAME, 'score').text
    except NoSuchElementException:
        pass

    # Try to extract the seller name from "seller-name__detail" within "seller-name"
    try:
        seller_name_element = driver.find_element(By.CLASS_NAME, 'seller-name__detail')
        seller_name = seller_name_element.text
    except NoSuchElementException:
        pass

    # Try to extract details from the "pdp-seller-info-pc" class
    try:
        info_contents = driver.find_elements(By.CLASS_NAME, 'info-content')
        if len(info_contents) >= 3:
            positive_seller_ratings = info_contents[0].find_element(By.CLASS_NAME, 'seller-info-value.rating-positive').text
            ship_on_time = info_contents[1].find_element(By.CLASS_NAME, 'seller-info-value').text
            chat_response_rate = info_contents[2].find_element(By.CLASS_NAME, 'seller-info-value').text
    except NoSuchElementException:
        pass

    try:
        comments.extend(extract_comments_from_page(driver))
        while True:
            # Locate the "Next Page" button using XPath
            next_page_button = driver.find_element(By.XPATH, '//li[@class="ant-pagination-next"]/button[@class="ant-pagination-item-link"]')
            print(next_page_button.is_displayed())
            while not next_page_button.is_displayed():
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

            if is_next_button_disabled(next_page_button):
              break

            next_page_button.click()
            time.sleep(3)


            comments.extend(extract_comments_from_page(driver))

    except NoSuchElementException:
        pass

    return {
        'mobile_name': mobile_name,
        'price': price,
        'rating': rating,
        'comments': comments,
        'seller_name': seller_name,
        'positive_seller_ratings': positive_seller_ratings,
        'ship_on_time': ship_on_time,
        'chat_response_rate': chat_response_rate
    }

In [40]:
# Create empty lists to store details
links_list = []
mobile_names = []
prices = []
ratings = []
comments = []
seller_names = []
positive_seller_ratings = []
ship_on_time = []
chat_response_rates = []

In [41]:
all_links = np.array(df['link'])
# Iterate through each link and get details
# Create an empty DataFrame to store details
details_df = pd.DataFrame(columns=[
    'Link', 'Mobile Name', 'Price', 'Rating',
    'Comments', 'Seller Name', 'Positive Seller Ratings',
    'Ship on Time', 'Chat Response Rate'
])
driver = web_driver()
for link in all_links:
    mobile_details = get_mobile_details(driver, link)

    # Print or use the details as needed
    print(link)
    # Print or use the details as needed
    print("Mobile Name:", mobile_details['mobile_name'])
    print("Price:", mobile_details['price'])
    print("Rating:", mobile_details['rating'])
    print("Comments:", mobile_details['comments'])
    print("Seller Name:", mobile_details['seller_name'])
    print("Positive Seller Ratings:", mobile_details['positive_seller_ratings'])
    print("Ship on Time:", mobile_details['ship_on_time'])
    print("Chat Response Rate:", mobile_details['chat_response_rate'])
    print("\n")
        # Append details to the lists
    links_list.append(link)
    mobile_names.append(mobile_details['mobile_name'])
    prices.append(mobile_details['price'])
    ratings.append(mobile_details['rating'])
    comments.append(mobile_details['comments'])
    seller_names.append(mobile_details['seller_name'])
    positive_seller_ratings.append(mobile_details['positive_seller_ratings'])
    ship_on_time.append(mobile_details['ship_on_time'])
    chat_response_rates.append(mobile_details['chat_response_rate'])


details_dict = {
    'Link': links_list,
    'Mobile Name': mobile_names,
    'Price': prices,
    'Rating': ratings,
    'Comments': comments,
    'Seller Name': seller_names,
    'Positive Seller Ratings': positive_seller_ratings,
    'Ship on Time': ship_on_time,
    'Chat Response Rate': chat_response_rates
}

# Convert the dictionary into a DataFrame
details_df = pd.DataFrame.from_dict(details_dict)

https://www.daraz.pk/products/y02-t-4-gb-64-gb-5-mp-8-mp-i427595776-s2031601043.html?search=1
Mobile Name: Vivo Y02 T RAM 4 GB ROM 64 GB Front Camera 5 MP Back Camera 8 MP Battery Type 5000 mAh
Price: Rs. 39,890
Rating: None
Comments: []
Seller Name: Sid Sam Communication Khanewal
Positive Seller Ratings: 93%
Ship on Time: 92%
Chat Response Rate: 98%


https://www.daraz.pk/products/6gb-6gb-y17s-128-664-5000mah-i434277493-s2082823249.html?search=1
Mobile Name: Vivo Y17s - 6GB + 6GB Expendable RAM 128 ROM - 6.64 Inch Display - 5000mAh Battery
Price: Rs. 45,499
Rating: 4.0
Comments: ['good 💯😊']
Seller Name: Kashif Electronics (Karachi)
Positive Seller Ratings: 89%
Ship on Time: 93%
Chat Response Rate: 100%


https://www.daraz.pk/products/a14-6gb-128-gb-pta-1-i428260915-s2035598256.html?search=1
Mobile Name: Samsung Galaxy A14 - 6GB ROM 128 GB - PTA Approved, 1 Year Official Warranty
Price: Rs. 57,199
Rating: None
Comments: []
Seller Name: Mobile Arena (Karachi)
Positive Seller Ratings: 89

In [42]:
details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Link                     890 non-null    object
 1   Mobile Name              873 non-null    object
 2   Price                    873 non-null    object
 3   Rating                   290 non-null    object
 4   Comments                 890 non-null    object
 5   Seller Name              873 non-null    object
 6   Positive Seller Ratings  854 non-null    object
 7   Ship on Time             854 non-null    object
 8   Chat Response Rate       851 non-null    object
dtypes: object(9)
memory usage: 62.7+ KB


In [44]:
details_df.head(10)

,Link,Mobile Name,Price,Rating,Comments,Seller Name,Positive Seller Ratings,Ship on Time,Chat Response Rate
0,https://www.daraz.pk/products/y02-t-4-gb-64-gb...,Vivo Y02 T RAM 4 GB ROM 64 GB Front Camera 5 M...,"Rs. 39,890",None,[],Sid Sam Communication Khanewal,93%,92%,98%
1,https://www.daraz.pk/products/6gb-6gb-y17s-128...,Vivo Y17s - 6GB + 6GB Expendable RAM 128 ROM -...,"Rs. 45,499",4.0,[good 💯😊],Kashif Electronics (Karachi),89%,93%,100%
2,https://www.daraz.pk/products/a14-6gb-128-gb-p...,Samsung Galaxy A14 - 6GB ROM 128 GB - PTA Appr...,"Rs. 57,199",None,[],Mobile Arena (Karachi),89%,46%,91%
3,https://www.daraz.pk/products/10-4gb-128gb-hd-...,"Tecno Spark 10 - 4GB RAM 128GB ROM - 6.6"" HD+9...","Rs. 33,499",None,[],Kashif Electronics (Karachi),89%,93%,100%
4,https://www.daraz.pk/products/20-i-4-gb-128-8-...,Infinix Hot 20 i RAM 4 GB ROM 128 Front Camera...,"Rs. 42,000",None,[],Atta Communication Khanewal,94%,100%,100%
5,https://www.daraz.pk/products/y-16-4-gb-64-gb-...,Vivo Y 16 RAM 4 GB ROM 64 GB Camera Front 5 MP...,"Rs. 39,990",None,[],ALSAEED COMMUNICATION KHANEWAL,95%,70%,97%
6,https://www.daraz.pk/products/4gb-a04-64gb-50m...,"Samsung A04 , 4GB RAM 64GB ROM , 50MP Main Cam...","Rs. 39,399",None,[],Mobile Dot,88%,60%,59%
7,https://www.daraz.pk/products/8gb-a24-1-000-gb...,Samsung A24 - 8GB RAM 128GB ROM - 6.6 Inches -...,"Rs. 79,999",None,[],Kashif Electronics (Karachi),89%,93%,100%
8,https://www.daraz.pk/products/5-lte-652-2-gb-3...,Tecno Spark 7t 4GB RAM 64 GB ROM Dual Sim 1 Ye...,"Rs. 36,999",5.0,"[highly satisfied, V.nice awesome mobile in re...",AL-KHAIR TRADING PAKISTAN,96%,100%,Not enough data
9,https://www.daraz.pk/products/tecno-spark-8c-6...,"Tecno Spark 8C - 6.6"" Inch Display - 4GB RAM -...","Rs. 30,590",None,[],New All In One,78%,95%,45%


In [45]:
details_df.to_csv("smartphones_data.csv", index=False)